<a href="https://colab.research.google.com/github/drscghosh/Testing/blob/master/DiallelGriffingModel1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.formula.api import ols

# Simulated diallel data (Griffing's Method I: parents + F1s + reciprocals)
data = {
    'Parent1': ['P1', 'P1', 'P1', 'P1', 'P2', 'P2', 'P2', 'P3', 'P3', 'P4'],
    'Parent2': ['P1', 'P2', 'P3', 'P4', 'P2', 'P3', 'P4', 'P3', 'P4', 'P4'],
    'Yield':   [20.5, 22.1, 23.4, 21.7, 19.9, 24.0, 22.5, 20.8, 21.9, 20.3]
}

df = pd.DataFrame(data)

# Include reciprocals (except diagonal/self-crosses)
reciprocals = df[df['Parent1'] != df['Parent2']].copy()
reciprocals['Parent1'], reciprocals['Parent2'] = reciprocals['Parent2'], reciprocals['Parent1']
reciprocals['Yield'] += np.random.normal(0, 0.5, size=len(reciprocals))  # Small noise

df = pd.concat([df, reciprocals], ignore_index=True)

# Model for ANOVA (fixed effects)
# Remove the 'Cross' term as it's redundant with Parent1 and Parent2,
# leading to multicollinearity and the ValueError.
# The interaction term C(Parent1):C(Parent2) captures SCA and reciprocal effects.
model = ols('Yield ~ C(Parent1) + C(Parent2) + C(Parent1):C(Parent2)', data=df).fit()
anova_table = sm.stats.anova_lm(model, type=2)

print("ANOVA Table:")
print(anova_table)

# Estimate GCA, SCA, Reciprocal effects
# This part estimates combining abilities manually
grand_mean = df['Yield'].mean()
parents = sorted(set(df['Parent1']) | set(df['Parent2']))
n_parents = len(parents)

# Calculate GCA effects
gca = {}
for p in parents:
    related_yields = df[(df['Parent1'] == p) | (df['Parent2'] == p)]['Yield']
    # Need to adjust this calculation based on the model output if you want to
    # align with the ANOVA results. For a simple mean-based GCA, this is okay,
    # but statsmodels would typically give estimates relative to a reference parent.
    # For this example, keeping the manual calculation as per original code.
    gca[p] = related_yields.mean() - grand_mean

# Estimate SCA and Reciprocal effects
sca = {}
recip = {}
# Assuming a balanced design for this simple loop, though manual calculations
# can be complex for unbalanced data or different Griffing methods.
for index, row in df.iterrows():
    p1, p2 = row['Parent1'], row['Parent2']
    y = row['Yield']

    if p1 != p2:
        # This calculation method for SCA and reciprocal effects is a simplification
        # and may not align perfectly with the ANOVA interaction term results,
        # especially with unbalanced data or certain model parameterizations.
        # A more rigorous method would involve extracting coefficients from the OLS model.
        # Keeping the manual calculation for demonstration as per original code structure.

        # Check if both reciprocal crosses exist before calculating reciprocal effect
        reciprocal_cross_exists = ((df['Parent1'] == p2) & (df['Parent2'] == p1)).any()

        if reciprocal_cross_exists:
             # This is a basic way to estimate the residual after removing GCA effects.
             # It's a simplification of SCA.
             gca_total = gca.get(p1, 0) + gca.get(p2, 0) # Use .get for safety
             expected = grand_mean + gca_total
             sca[(p1, p2)] = y - expected

             # Calculate reciprocal effect only if the reciprocal cross is present
             reciprocal_yield = df[(df['Parent1'] == p2) & (df['Parent2'] == p1)]['Yield'].iloc[0]
             recip[(p1, p2)] = y - reciprocal_yield


# Print results (using sorted keys for consistent output)
print("\nGeneral Combining Ability (GCA):")
for p in sorted(gca.keys()):
    print(f"{p}: {gca[p]:.2f}")

print("\nSpecific Combining Ability (SCA):")
# Sort SCA keys for consistent output
for k in sorted(sca.keys()):
    print(f"{k[0]} x {k[1]}: {sca[k]:.2f}")

print("\nReciprocal Effects:")
# Sort Reciprocal keys for consistent output
for k in sorted(recip.keys()):
    # Only print one direction of the reciprocal difference (P1xP2 vs P2xP1)
    if (k[1], k[0]) not in recip or k[0] < k[1]: # Prevent double printing and enforce order
         if k in recip and (k[1], k[0]) in recip: # Ensure both directions exist to show difference
            print(f"{k[0]} x {k[1]} vs {k[1]} x {k[0]}: {recip[k]:.2f}")
         elif k in recip: # Handle cases where only one direction is in the recip dict (due to calculation logic)
             print(f"{k[0]} x {k[1]}: {recip[k]:.2f}")

ANOVA Table:
                        df        sum_sq   mean_sq    F  PR(>F)
C(Parent1)             3.0  1.891864e+00  0.630621  0.0     NaN
C(Parent2)             3.0  2.061333e+00  0.687111  0.0     NaN
C(Parent1):C(Parent2)  9.0  1.663455e+01  1.848284  0.0     NaN
Residual               0.0  2.587464e-27       inf  NaN     NaN

General Combining Ability (GCA):
P1: 0.21
P2: 0.34
P3: 0.64
P4: -0.28

Specific Combining Ability (SCA):
P1 x P2: -0.43
P1 x P3: 0.57
P1 x P4: -0.20
P2 x P1: 0.25
P2 x P3: 1.04
P2 x P4: 0.47
P3 x P1: 0.26
P3 x P2: 0.22
P3 x P4: -0.43
P4 x P1: -0.17
P4 x P2: -0.28
P4 x P3: -0.39

Reciprocal Effects:
P1 x P2 vs P2 x P1: -0.68
P1 x P3 vs P3 x P1: 0.32
P1 x P4 vs P4 x P1: -0.03
P2 x P3 vs P3 x P2: 0.83
P2 x P4 vs P4 x P2: 0.75
P3 x P4 vs P4 x P3: -0.04


/usr/local/lib/python3.11/dist-packages/statsmodels/stats/anova.py:142: RuntimeWarning: divide by zero encountered in scalar divide
  (model.ssr / model.df_resid))
